<a href="https://colab.research.google.com/github/fenikowski/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Plantilla_Seminario_Algoritmos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Seminario<br>
Nombre y Apellidos: Igor Fenikowski  <br>
Url: https://github.com/fenikowski/03MAIR--Algoritmos-de-Optimizacion--2020/blob/master/Seminario/Plantilla_Seminario_Algoritmos.ipynb<br>
Url Colab: https://colab.research.google.com/drive/1gD2QCehUReSlIX9qZIN4GmxLu7Kk25_d?usp=sharing<br>
Problema:
>2. Organizar los horarios de partidos de La Liga<br>

Descripción del problema:

>Desde la La Liga de fútbol profesional se pretende organizar los horarios de los partidos de
liga de cada jornada. Se conocen algunos datos que nos deben llevar a diseñar un
algoritmo que realice la asignación de los partidos a los horarios de forma que maximice la audiencia. <br>
>Los horarios disponibles se conocen a priori y son los siguientes: <br>
> - Viernes: 20 <br>
> - Sábado: 12, 16, 18, 20 <br>
> - Domingo: 12, 16, 18, 20 <br>
> - Lunes: 20 <br>

>En primer lugar se clasifican los equipos en tres categorías según el numero de
seguidores( que tiene relación directa con la audiencia). Hay 3 equipos en la
categoría A, 11 equipos de categoría B y 6 equipos de categoría C.
Se conoce estadisticamente la audiencia que genera cada partido según los equipos que se enfrentan y en horario de sábado a las 20h (el mejor en todos los casos). Si el horario del partido no se realiza a las 20 horas del sábado se sabe que se reduce. Debemos asignar obligatoriamente siempre un partido el viernes y un partido el lunes. Es posible la coincidencia de horarios pero en este caso la audiencia de cada partido se verá afectada y se estima que se reduce en porcentaje.


                                

In [1]:
# Importación de librerías necesarias

import pandas as pd
import copy
import random
import math
from itertools import combinations

(*)¿Cuantas posibilidades hay sin tener en cuenta las restricciones?<br>



¿Cuantas posibilidades hay teniendo en cuenta todas las restricciones.




Respuesta

In [2]:
# Para cada partido hay 10 posibilidades de asignar una fecha por tanto hay 10 elevado a 10 posibilidades en total.

print(10 ** 10)

10000000000


In [3]:
# Ahora teniendo en cuenta las restricciones (de al menos un partido el viernes y el lunes):

# Supongamos que A es un evento aleatorio en el cual hay al menos un partido el viernes y el lunes.
# Supongamos también que A' es un evento aleatorio en el cual el viernes y el lunes no tienen ningún partido asignado.

# Basando se en el teorema de la probabilidad total obtenemos que: 

$$ A + A' = Ω  \iff A = Ω - A'$$

In [4]:
# Calculemos el evento A'. Siguiendo la misma logica que en el apartado anterior, podemos asignar cada partido a una de las ocho fechas, lo que nos da 10^8 posibilidades.
# Teniendo calculadas las posibilidades en eventos A' y Ω, podemos al final avanzar al último paso, que es calcular posibilidades del evento A:

$$ A = 10^{10} - 10^8 = 10^8 * (10^2 - 1) = 10^8 * 99 $$

In [5]:
print((10 ** 8) * 99)

9900000000


Modelo para el espacio de soluciones<br>
(*) ¿Cual es la estructura de datos que mejor se adapta al problema? Argumentalo.(Es posible que hayas elegido una al principio y veas la necesidad de cambiar, arguentalo)


Respuesta

In [6]:
# He elegido modelar los datos del problema en varios diccionarios y arrays para poder facilmente aplicar "penalizaciones" por coindencia o hora, 
# tanto como poder conseguir información clara sobre equipos disponibles y calendario creado.

# Partidos para los que hay que preparar la jornada:
# Para poder manejar bien datos de los partidos, he decido preparar un objeto con información sobre cada uno (equipos y categoría) y meterlos en una lista.
# Esto me permitirá facilmente tanto iterar sobre ellos, como quitar partidos con fecha ya asignada.
games = [
    { "teams": "Real Madrid - Celta", "category": "AB"},
    { "teams": "R. Sociedad - Valencia", "category": "AB"},
    { "teams": "Mallorca - Eibar", "category": "CC"},
    { "teams": "Barcelona - Athletic", "category": "AB"},
    { "teams": "Leganés - Osasuna", "category": "CC"},
    { "teams": "Villareal - Granada", "category": "BC"},
    { "teams": "Alavés - Levante", "category": "BB"},
    { "teams": "Espanyol - Sevilla", "category": "BB"},
    { "teams": "Betis - Valladolid", "category": "BC"},
    { "teams": "Atlético - Getafe", "category": "BB"},
]

# Diccionario con audiencia que generan partidos entre diferentes equipos los sábados a las 20:
# (para poder facilmente sacar valor asignado a partido de cada categoría)
category_converter = {"AA":2, "AB":1.3, "AC":1, "BB":0.9, "BC":0.75, "CC":0.47}

# Lista con penelizaciones por organizar más que un partido a una hora determinada:
coincidences = [0, 1, 0.75, 0.55, 0.4, 0.3, 0.25, 0.22, 0.2, 0.2]

# He elegido un diccionario para poder ir guardando información sobre la jornada a lo largo de ejecución de algoritmos:
# - "weight" se refiere a la ponderación asignada a cada hora, 
# - audiencia total inicializada,
# - array de partidos inicializado

scheldule = {
    "V20": { "games": [], "weight": 0.4, "audience": 0 },
    "S12": { "games": [], "weight": 0.55, "audience": 0 },
    "S16": { "games": [], "weight": 0.7, "audience": 0 },
    "S18": { "games": [], "weight": 0.8, "audience": 0 },
    "S20": { "games": [], "weight": 1, "audience": 0 },
    "D12": { "games": [], "weight": 0.45, "audience": 0 },
    "D16": { "games": [], "weight": 0.75, "audience": 0 },
    "D18": { "games": [], "weight": 0.5, "audience": 0 },
    "D20": { "games": [], "weight": 1, "audience": 0 },
    "L20": { "games": [], "weight": 0.4, "audience": 0 }
}

Según el modelo para el espacio de soluciones<br>
(*)¿Cual es la función objetivo?

(*)¿Es un problema de maximización o minimización?

Respuesta

In [7]:
# Para calcular la audiencia para un partido hay que multiplicar la audiencia que éste genera el sabado a las 20 por ponderación de la fecha que le asignamos
# y penalización por coincidencia de partidos. La funcion objetivo podemos presentar por tanto como una suma de mencionado calculo ejecutado 
# para cada uno de los 10 partidos donde:
#
#   x - un partido
#   p - ponderación de la fecha elegida
#   c - penalizacion por coincidencia
#

$$ Max! = \sum\limits_{i=1}^{10} x_i * p * c $$

In [8]:
# Como se trata de obtener la jornada con mayor número de audiencia para 10 partidos jugados entre 20 equipos, es un problema de maximización.

Diseña un algoritmo para resolver el problema por fuerza bruta

Respuesta

In [9]:
# FUNCIONES AUXILIARES

# funcion que calcula la diferencia en audiencia (positiva o negativa) de jugar una partida hipotetica

def count_audience_difference(date, game, scheldule):
  games_audience = category_converter[game["category"]]
  new_audience = 0

  for game in scheldule[date]["games"]:
    games_audience += category_converter[game["category"]]

  new_audience = scheldule[date]["weight"] * coincidences[len(scheldule[date]["games"]) + 1] * games_audience

  return new_audience - scheldule[date]["audience"], new_audience



# funcion que sirve para crear una partida en el calendario:

def create_game(date, game, new_audience, scheldule):
  reference_scheldule = copy.deepcopy(scheldule)

  reference_scheldule[date]["games"].append(game)
  reference_scheldule[date]["audience"] = new_audience
  return reference_scheldule


# funcion que transforma el calendario en una lista, para poder facilmente crear un DataFrame

def transform_scheldule(data):
  result = []
  for date, values in data.items():
    for game in values["games"]:
      result.append({ 
          "Fecha": date, 
          "Partido": game["teams"], 
          "Category": game["category"][0] + " - " + game["category"][1], 
          "Base": category_converter[game["category"]],
          "Ponderación": values["weight"],
          "Base * Ponderación": category_converter[game["category"]] * values["weight"],
          "Correción coincidencia": round(values["weight"] * category_converter[game["category"]] * coincidences[len(values["games"])],2) 
          })
  return result

In [10]:
# Ya que la complejidad de algoritmo por fuerza bruta es muy alta, el calculo para propuestos datos de entrada es muy largo
# y no es posible obtener ningún resultado en tiempo razonable. Por tanto para testearlo y demostrar que si que podría conseguir buenos resultados,
# me he permitido reducir el número de partidos de la entrada (solo en este caso) a 6 partidos.

games = [
    { "teams": "Real Madrid - Celta", "category": "AB"},
    { "teams": "R. Sociedad - Celta", "category": "AB"},
    { "teams": "Mallorca - Eibar", "category": "CC"},
    { "teams": "Barcelona - Athletic", "category": "AB"},
    { "teams": "Leganés - Osasuna", "category": "CC"},
    { "teams": "Villareal - Granada", "category": "BC"}
]

In [11]:
# ALGORITMO POR FUERZA BRUTA

%%time

global_total_audience = 0
global_scheldule = {}

def brute_force(scheldule, games, total_audience = 0):
  global global_total_audience
  global global_scheldule

  # inicialización de variables
  reference_scheldule = scheldule.copy()
  reference_games = games.copy()
  reference_audience = total_audience

  # se saca un partido de la lista eliminandolo para que no se tome en cuenta en proximas iteraciones
  game = reference_games.pop(0)

  for date in reference_scheldule.keys():
    new_reference_scheldule = reference_scheldule.copy()
    new_reference_audience = reference_audience
    new_reference_games = reference_games.copy()
    
    # se calucula diferencia en audiencia proporcionada por nuevo partido
    new_difference, new_audience = count_audience_difference(date, game, reference_scheldule) 
    new_game = { "date": date, "game": game, "new_audience": new_audience }

    # los datos del nuevo partido se guardan en el calendario
    new_reference_scheldule = create_game(new_game["date"], new_game["game"], new_game["new_audience"], reference_scheldule)
    new_reference_audience += new_difference
    
    # si no hay mas partidos para asignar al calendario, el algoritmo se para y valora el resultado
    if len(new_reference_games) > 0:
      brute_force(new_reference_scheldule, new_reference_games, new_reference_audience)

    else:
      # si el resultado es mejor y hay partidos asignados el viernes y el lunes, se sobrescribe el calendario global, cual se devuelve al final del algoritmo
      if new_reference_audience > global_total_audience and len(new_reference_scheldule["L20"]["games"]) > 0 and len(new_reference_scheldule["V20"]["games"]) > 0:
        global_total_audience = new_reference_audience
        global_scheldule = new_reference_scheldule


brute_force(scheldule, games)

CPU times: user 1min 38s, sys: 18.5 ms, total: 1min 38s
Wall time: 1min 38s


In [12]:
frame = transform_scheldule(global_scheldule)
df = pd.DataFrame(frame)
df = df.append({"Fecha": "", "Partido": "", "Category": "", "Base": "", "Ponderación": "", "Base * Ponderación": "", "Correción coincidencia": df["Correción coincidencia"].sum()}, ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(global_total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partido,Category,Base,Ponderación,Base * Ponderación,Correción coincidencia
0,V20,Mallorca - Eibar,C - C,0.47,0.4,0.188,0.19
1,S18,Real Madrid - Celta,A - B,1.3,0.8,1.04,1.04
2,S20,R. Sociedad - Celta,A - B,1.3,1,1.3,1.30
3,D16,Villareal - Granada,B - C,0.75,0.75,0.5625,0.56
4,D20,Barcelona - Athletic,A - B,1.3,1,1.3,1.30
5,L20,Leganés - Osasuna,C - C,0.47,0.4,0.188,0.19
6,,,,,,,4.58


La audiencia total del calendario generado es igual a: 4.58 milliones de personas.


Calcula la complejidad del algoritmo por fuerza bruta

Respuesta

In [13]:
# Mi algoritmo por fuerza bruta se basa en recursividad, con cada partido por jugar, el número de operaciones crece exponencialmente.
# Al principio el algoritmo hace un bucle para meter el partido en cada fecha posible, y después de conseguirlo, se ejecuta otra vez para siguiente partido con la misma intención.
# La ejecución se repite haste que cada partido tenga una fecha asignada. Por tanto, si hay 2 equipos, habrá 10^2 operaciones.
# Si habrá 3 equipos, el número crecerá a 10^3. De dichos calculos se concluye que la complejidad del algoritmo de fuerza bruta es de orden exponencial.

(*)Diseña un algoritmo que mejore la complejidad del algortimo por fuerza bruta. Argumenta porque crees que mejora el algoritmo por fuerza bruta

Respuesta

In [14]:
# Para volver a problema original, inicializo otra vez el número de partidos, que anteriormente simplifiqué para testear el algoritmo de fuerza bruta.
games = [
    { "teams": "Real Madrid - Celta", "category": "AB"},
    { "teams": "R. Sociedad - Valencia", "category": "AB"},
    { "teams": "Mallorca - Eibar", "category": "CC"},
    { "teams": "Barcelona - Athletic", "category": "AB"},
    { "teams": "Leganés - Osasuna", "category": "CC"},
    { "teams": "Villareal - Granada", "category": "BC"},
    { "teams": "Alavés - Levante", "category": "BB"},
    { "teams": "Espanyol - Sevilla", "category": "BB"},
    { "teams": "Betis - Valladolid", "category": "BC"},
    { "teams": "Atlético - Getafe", "category": "BB"},
]

In [15]:
# Mi candidato para dar una solución buena en tiempo limitado es un algoritmo voraz. Un algoritmo voraz se basa en obtener la solución optima en cada paso con la esperanza
# de llegar a una solución general óptima. En este problema, un algoritmo voraz notablemente accelera el calcúlo y permite sacar un resultado bueno en tiempo razonable.

In [16]:
# ALGORITMO VORAZ

%%time

def greedy_algorithm(scheldule, games):

  # inicialización de variables
  reference_scheldule = copy.deepcopy(scheldule)
  reference_games = copy.deepcopy(games)
  total_audience = 0

  # un bucle para iterar sobre número total de partidas por jugar
  for index, game in enumerate(reference_games):

    refference_difference = -999
    new_game = {}

    # condición que permite asegurarse de que lunes y viernes tienen un partido asignado
    if index == len(games) and len(reference_scheldule["V20"]["games"]):
      new_difference, new_audience = count_audience_difference("V20", game, reference_scheldule) 

      refference_difference = new_difference
      new_game = { "date": "V20", "game": game, "new_audience": new_audience }


    elif index == len(games) + 1 and len(reference_scheldule["L20"]["games"]):
      new_difference, new_audience = count_audience_difference("L20", game, reference_scheldule) 

      refference_difference = new_difference
      new_game = { "date": "L20", "game": game, "new_audience": new_audience }


    else:
      # un bucle para iterar por todas las fechas disponibles
      for date in reference_scheldule.keys():

        # para cada fecha se calcula la diferencia en audiencia en caso de crear una partida, 
        # teniendo en cuenta audiencia de la fecha, penalización por coincidencia para todas partidas de la fecha y audiencia por categoría de la partida
        # (la diferencia puede tener valor negativo)

        new_difference, new_audience = count_audience_difference(date, game, reference_scheldule) 

        # si la diferencia es mayor que la de referencia, se guarda el nuevo valor de diferencia y detalles del partido
        if(new_difference > refference_difference):
          refference_difference = new_difference
          new_game = { "date": date, "game": game, "new_audience": new_audience }

    # al final del bucle se crea el partido que atrae el mayor número de aficionados y se guarda la nueva audiencia total
    reference_scheldule = create_game(new_game["date"], new_game["game"], new_game["new_audience"], reference_scheldule)
    total_audience += refference_difference

  return total_audience, reference_scheldule


total_audience, created_scheldule = greedy_algorithm(scheldule, games)

CPU times: user 454 µs, sys: 1.18 ms, total: 1.64 ms
Wall time: 1.51 ms


In [17]:
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append({"Fecha": "", "Partido": "", "Category": "", "Base": "", "Ponderación": "", "Base * Ponderación": "", "Correción coincidencia": df["Correción coincidencia"].sum()}, ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partido,Category,Base,Ponderación,Base * Ponderación,Correción coincidencia
0,S12,Villareal - Granada,B - C,0.75,0.55,0.4125,0.41
1,S16,Leganés - Osasuna,C - C,0.47,0.7,0.329,0.25
2,S16,Atlético - Getafe,B - B,0.9,0.7,0.63,0.47
3,S18,Mallorca - Eibar,C - C,0.47,0.8,0.376,0.28
4,S18,Espanyol - Sevilla,B - B,0.9,0.8,0.72,0.54
5,S20,Real Madrid - Celta,A - B,1.3,1,1.3,1.30
6,D12,Betis - Valladolid,B - C,0.75,0.45,0.3375,0.34
7,D16,Barcelona - Athletic,A - B,1.3,0.75,0.975,0.98
8,D18,Alavés - Levante,B - B,0.9,0.5,0.45,0.45
9,D20,R. Sociedad - Valencia,A - B,1.3,1,1.3,1.30


La audiencia total del calendario generado es igual a: 6.32 milliones de personas.


(*)Calcula la complejidad del algoritmo 

Respuesta

In [18]:
# Primero el algoritmo ejecuta un bucle por cada partido que se desea poner en la jornada. Lo siguiente es ejecutar un bucle por todas las fechas posibles (para cada partido).
# Entonces para dar un ejemplo, si hay que asignarles fechas a dos partidos, el algoritmo hará un bucle por cada uno (2) para luego ejecutar en todos dos casos otro bucle por fechas (10),
# que resulta en darnos 2 * 10 operaciones. Siguiendo la misma logica, para tres equipos obtendremos 3 * 10, y para cuatro 4 * 10. Esto quiere decir que la complejidad de algoritmo,
# es de orden lineal, ya que por cada partido más al número de operaciones total se añaden 10 operaciones más.

Según el problema (y tenga sentido), diseña un juego de datos de entrada aleatorios

Respuesta

In [19]:
# Lista con equipos disponibles
teams = [
         { "name": "Real Madrid", "category": "A"},
         { "name": "Barcelona", "category": "A"},
         { "name": "R. Sociedad", "category": "A"},
         { "name": "Atlético", "category": "B"},
         { "name": "Getafe", "category": "B"},
         { "name": "Espanyol", "category": "B"},
         { "name": "Sevilla", "category": "B"},
         { "name": "Athletic", "category": "B"},
         { "name": "Celta", "category": "B"},
         { "name": "Valencia", "category": "B"},
         { "name": "Betis", "category": "B"},
         { "name": "Alavés", "category": "B"},
         { "name": "Levante", "category": "B"},
         { "name": "Villareal", "category": "B"},
         { "name": "Granada", "category": "C"},
         { "name": "Valladolid", "category": "C"},
         { "name": "Osasuna", "category": "C"},
         { "name": "Leganés", "category": "C"},
         { "name": "Mallorca", "category": "C"},
         { "name": "Eibar", "category": "C"}
]

# Funcion para sacar 10 partidos entre equipos proporcionados
def create_games(teams):
  games = []
  random.shuffle(teams)
  pairs = [teams[i::10] for i in range(10)]
  for pair in pairs:
    game = pair[0]["name"] + ' - ' + pair[1]["name"]
    categories = [pair[0]["category"], pair[1]["category"]]
    categories.sort()
    category = categories[0] + categories[1]
    games.append({ "teams": game, "category": category })

  return games

games = create_games(teams)
df = pd.DataFrame(games)
display(df)

,teams,category
0,Betis - Celta,BB
1,Athletic - Levante,BB
2,Valencia - Alavés,BB
3,Barcelona - R. Sociedad,AA
4,Mallorca - Real Madrid,AC
5,Eibar - Sevilla,BC
6,Espanyol - Villareal,BB
7,Valladolid - Granada,CC
8,Atlético - Getafe,BB
9,Osasuna - Leganés,CC


Aplica el algoritmo al juego de datos generado

Respuesta

In [20]:
# ALGORITMO VORAZ

total_audience, created_scheldule = greedy_algorithm(scheldule, games)
frame = transform_scheldule(created_scheldule)
df = pd.DataFrame(frame)
df = df.append({"Fecha": "", "Partido": "", "Category": "", "Base": "", "Ponderación": "", "Base * Ponderación": "", "Correción coincidencia": df["Correción coincidencia"].sum()}, ignore_index=True)
df.fillna('',inplace=True)
display(df)
print("La audiencia total del calendario generado es igual a:", '\033[1m' + str(round(total_audience,2)) + '\033[0m', "milliones de personas.")

,Fecha,Partido,Category,Base,Ponderación,Base * Ponderación,Correción coincidencia
0,S12,Eibar - Sevilla,B - C,0.75,0.55,0.4125,0.41
1,S16,Mallorca - Real Madrid,A - C,1,0.7,0.7,0.70
2,S18,Valencia - Alavés,B - B,0.9,0.8,0.72,0.72
3,S20,Betis - Celta,B - B,0.9,1,0.9,0.68
4,S20,Espanyol - Villareal,B - B,0.9,1,0.9,0.68
5,D12,Osasuna - Leganés,C - C,0.47,0.45,0.2115,0.21
6,D16,Barcelona - R. Sociedad,A - A,2,0.75,1.5,1.50
7,D18,Valladolid - Granada,C - C,0.47,0.5,0.235,0.23
8,D20,Athletic - Levante,B - B,0.9,1,0.9,0.68
9,D20,Atlético - Getafe,B - B,0.9,1,0.9,0.68


La audiencia total del calendario generado es igual a: 6.48 milliones de personas.


Enumera las referencias que has utilizado(si ha sido necesario) para llevar a cabo el trabajo

Respuesta

In [21]:
# No he utilizado ningun otro fuente aparte del manual de la asignatura

Describe brevemente las lineas de como crees que es posible avanzar en el estudio del problema. Ten en cuenta incluso posibles variaciones del problema y/o variaciones al alza del tamaño

Respuesta